In [1]:
using Revise
using GLMakie, LinearAlgebra, StaticArrays, BenchmarkTools, Statistics, Distributions, ProgressMeter
using CUDA

includet("cuda.jl");
const SVec2 = SVector{2, Float32};

In [2]:
N = 10000
positions = 10 .* (rand(SVec2, N) .- Ref(SA[0.5f0,0.5f0]));
positions = cu(positions);
velocities = zero(positions);
fx = CUDA.zeros(N); fy = CUDA.zeros(N);

In [3]:
fig = Figure(resolution = (1800, 1800)); ax = Axis(fig[1,1], xticks = -10:1:10, yticks = -10:1:10)
node = Observable(Array(positions))
scatter!(ax, node, markersize = 10)
r = maximum(first, positions)
# xlims!(ax, -1.5r, 1.5r)
# ylims!(ax, -1.5r, 1.5r)
r = Observable(1.1maximum(first, positions))
on(node) do _
    r[] = 0.98r[] + 0.02*1.1maximum(first, positions)
    xlims!(ax, -r[], r[])
    ylims!(ax, -r[], r[])
end
# arrows!(ax, first.(Array(positions)), last.(Array(positions)), Array(fx) ./ 1000, Array(fy) ./ 1000, arrowsize = 5)
display(fig)

GLMakie.Screen(...)

In [442]:
for _ in 1:10000
    for _ in 1:10
        cellinds, perm, firsts = sortbycells(positions)
        positions = positions[perm]
        velocities = velocities[perm]
        force!(fx, fy, positions, cellinds, firsts)
        velocities .+= SVec2.(fx, fy) ./ 10000
        velocities .-= 0.0005.*positions

        T = 0.5mean(v -> dot(v, v), velocities)
        dT = 1e-2 * (0.4 - T)
        velocities .*= sqrt((T + dT) / T)

        positions .+= velocities ./ 1000
    end
    ispressed(fig, Keyboard.escape) && break
    node[] = Array(positions)
end

In [171]:
@benchmark count!($counts, $positions, $cellinds, $firsts)

BenchmarkTools.Trial: 1797 samples with 7 evaluations.
 Range (min … max):    3.714 μs …   4.173 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      13.829 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   397.560 μs ± 837.327 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                   ▁▁▁▂▁      
  █▇▃▁▄▃▃▃▅▅▁▄▄▅▃▄▁▃▄▁▄▃▁▁▄▃▁▁▃▄▁▁▁▃▃▁▁▁▃▁▁▁▄▁▃▁▃▃▅▁▅▇███████▅▆ █
  3.71 μs       Histogram: log(frequency) by time       2.42 ms <

 Memory estimate: 1.56 KiB, allocs estimate: 27.

In [132]:
c = 0
P = Array(positions)
for i in 1:length(P), j in 1:length(P)
    if norm(P[i] - P[j]) < 1
        c += 1
    end
end
c

4334

In [168]:
P = Array(positions)
C = map(1:length(P)) do i
    c = 0
    for j in 1:length(P)
        if norm(P[i] - P[j]) < 1
            c += 1
        end
    end
    c
end
C

10000-element Vector{Int64}:
 33
 34
 42
 35
 35
 42
 43
 42
 37
 40
  ⋮
 36
 43
 36
 43
 38
 44
 36
 35
 50

In [1]:
1

1

In [ ]:

Y = cu(X)
@time sort!(X, by = morton);
CUDA.@time sort!(Y, by = morton);

In [74]:
Int.(Array(morton.(Y)))

10000-element Vector{Int64}:
    1
    1
    1
    1
    1
    2
    2
    2
    2
    3
    3
    3
    3
    3
    3
    4
    4
    5
    6
    6
    ⋮
 3841
 3841
 3841
 3841
 3841
 3842
 3842
 3842
 3842
 3842
 3842
 3842
 3843
 3843
 3844
 3844
 3844
 3844
 3844
 3844

In [208]:
function cu_plot(positions; T=Float32, N=1024, resolution=(800, 600))
    fig = Figure(; resolution)
    ax = Axis(fig[1, 1]; limits=(0, 1, 0, 1))
    screen = display(fig)

    buffer = GLMakie.GLAbstraction.GLBuffer(Point2f, N)
    resource = let
        ref = Ref{CUDA.CUgraphicsResource}()
        CUDA.cuGraphicsGLRegisterBuffer(ref, buffer.id, CUDA.CU_GRAPHICS_MAP_RESOURCE_FLAGS_WRITE_DISCARD)
        ref[]
    end

    CUDA.cuGraphicsMapResources(1, [resource], stream())

    # get a CuArray object that we can work with
    array = let
        ptr_ref = Ref{CUDA.CUdeviceptr}()
        numbytes_ref = Ref{Csize_t}()
        CUDA.cuGraphicsResourceGetMappedPointer_v2(ptr_ref, numbytes_ref, resource)

        ptr = reinterpret(CuPtr{Point2f}, ptr_ref[])
        len = Int(numbytes_ref[] ÷ sizeof(Point2f))

        unsafe_wrap(CuArray, ptr, len)
    end

    broadcast!(array, positions) do p
        return Point2f(p...)
    end

    synchronize()

    CUDA.cuGraphicsUnmapResources(1, [resource], stream())

    scatter!(ax, buffer)

    GLMakie.render_frame(screen; resize_buffers=false)
    GLMakie.glFinish()

    CUDA.cuGraphicsUnregisterResource(resource)

    return
end

cu_plot (generic function with 2 methods)

In [230]:
screen = display(scatter(rand(SVec2, 1024)))
propertynames(screen)

(:glscreen, :shader_cache, :framebuffer, :config, :stop_renderloop, :rendertask, :screen2scene, :screens, :renderlist, :postprocessors, :cache, :cache2plot, :framecache, :render_tick, :window_open, :root_scene, :reuse, :close_after_renderloop, :requires_update)

In [243]:
screen.framebuffer.buffers

Dict{Symbol, GLMakie.GLAbstraction.Texture} with 6 entries:
  :color      => Texture2D, ID: 1, Size: (800, 600)…
  :OIT_weight => Texture2D, ID: 5, Size: (800, 600)…
  :HDR_color  => Texture2D, ID: 4, Size: (800, 600)…
  :depth      => Texture2D, ID: 3, Size: (800, 600)…
  :objectid   => Texture2D, ID: 2, Size: (800, 600)…
  :stencil    => Texture2D, ID: 3, Size: (800, 600)…

In [248]:
propertynames(screen.screens[3][2])

(:parent, :events, :px_area, :clear, :camera, :camera_controls, :transformation, :plots, :theme, :children, :current_screens, :backgroundcolor, :visible, :ssao, :lights, :deregister_callbacks)